# The Plan
Everything we know about a specific plan. 

### Instructions
1. Run all cells! (click on Menu > Cell > Run All Cells)
1. View report at the bottom.

In [ ]:
planId = "e379b1c6-ffe1-4d1f-9c85-841e15a87631"

In [ ]:
%%capture 

# install packages, setup workspace root
!pip install azure-kusto-notebooks matplotlib
import os
import sys
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import concurrent.futures
from azure.kusto.notebooks import utils as akn

# cwd should be workspace root
if os.path.basename(os.getcwd()) == 'devops-pipelines':
    os.chdir(os.pardir)

In [ ]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
client = akn.get_client('https://vso.kusto.windows.net')

In [ ]:
# DEBUG: reload akn module
# import importlib
# importlib.reload(akn)

In [ ]:
# collect basic plan info
plan_info = akn.Query(
    client, 'VSO', 
    path=os.path.join('devops-pipelines', 'queries', 'run', 'PlanInfo.csl'), 
    params={'OrchestrationId': akn.quote(planId)})

# collect full plan history
what_happened = akn.Query(client, 'VSO',
    path=os.path.join('devops-pipelines', 'queries', 'run', 'WhatHappened.csl'),
    params={'OrchestrationId': akn.quote(planId)})

# fetch all data in parallel
akn.run((plan_info, what_happened))

In [ ]:
# draw basic info

# compute relative time stamps
history = what_happened.dataframe
t0 = history['PreciseTimeStamp'].iloc[0]
history['Time'] = history.apply(lambda row: row['PreciseTimeStamp'] - t0, axis=1)
history.OrchestrationId = history.apply(lambda row: row.OrchestrationId[37:], axis=1)

# record critical times
def find_time(message):
    r = history[history.Message.str.startswith(message)]
    if len(r.index) > 0:
        return r['PreciseTimeStamp'].iloc[0]

create_time = find_time('Created plan')
start_time = find_time('Started plan')
end_time = find_time('Completed orchestration with result')
total_duration = end_time-start_time
d = akn.pandas_row_to_dictionary(plan_info.dataframe)
d['create time'] = create_time
d['start time'] = start_time
d['end time'] = end_time
d['total duration'] = total_duration

r = akn.Report()
r.write('## Plan was created by "', d['Name'], '" at ', create_time, ' and ran for ', total_duration, '.')
r.write(akn.to_md_table(d))

from IPython.display import Markdown
Markdown(r.content)

In [ ]:
# draw full history
columns_to_ignore = ('source_', 'PreciseTimeStamp')
columns = ['Time'] + [c for c in history.columns if c not in columns_to_ignore and c != 'Time']
pd.set_option('max_colwidth', 200)

from IPython.display import HTML
HTML(history[columns].to_html(index=False))